# 1.1.3 Post-processing Qbox Outputs to compute dynamical matrix.
Here we would learn how to post-process the qbox outputs after all the jobs regarding the displaced coordinates have finished. Here the qbox output files are fd-1.r and fd-2.r.

For that purpose we need to import the following objects from pyepfd.

In [1]:
from pyepfd.coord_util import *
from pyepfd.elph_classes import *
from pyepfd.pyepfd_io import *

                                                 
          ███████████                            
         ░░███░░░░░███                           
          ░███    ░███ █████ ████                
          ░██████████ ░░███ ░███                 
          ░███░░░░░░   ░███ ░███                 
          ░███         ░███ ░███                 
          █████        ░░███████                 
         ░░░░░          ░░░░░███                 
                        ███ ░███                 
                       ░░██████                  
                        ░░░░░░                   
 ██████████ ███████████  ███████████ ██████████  
░░███░░░░░█░░███░░░░░███░░███░░░░░░█░░███░░░░███ 
 ░███  █ ░  ░███    ░███ ░███   █ ░  ░███   ░░███
 ░██████    ░██████████  ░███████    ░███    ░███
 ░███░░█    ░███░░░░░░   ░███░░░█    ░███    ░███
 ░███ ░   █ ░███         ░███  ░     ░███    ███ 
 ██████████ █████        █████       ██████████  
░░░░░░░░░░ ░░░░░        ░░░░░       ░░░░░░░░░░   


First, we would read the qbox outputs and store them within a list object. Let us first initialize the list object named qbouts.

In [2]:
qbouts = []

Now we would read the qbox outputs using the qbox class available within coord_utils.

In [3]:
for i in range(2):
    qbouts.append(qbox(file_path = 'fd-'+str(i+1)+'.r',io = 'r'))

Time spent on qbox class: 0.02070307731628418 s.
Time spent on qbox class: 0.018831729888916016 s.


We are running the loop twice as we have only two outputs. Change it according to your needs. qbox class have objects that stores coordinates (coords), atoms, mass, forces etc. Now we would store them into separate variables from the list of qbox objects. For that purpose first we initialize these objects with the first element of the qbouts. 

In [4]:
coords = qbouts[0].coords
atoms = qbouts[0].atoms
mass = qbouts[0].mass
forces = qbouts[0].forces

Next we will append other elements upto the length of the qbouts.

In [5]:
for i in range(1,len(qbouts)):
    coords = np.vstack((coords,qbouts[i].coords))
    forces = np.vstack((forces,qbouts[i].forces))

The optimized geometry is the first set of coordinates. So we define it accordingly,

In [6]:
opt_coord = coords[0]

The cell parameters are fixed. Therefore we can take the first cell_parameters and conver them into vectors using abc2h function available within pyepfd.coord_utils.

In [7]:
cell_v = abc2h(qbouts[0].cell)

Now we compute the force constant matrix and diagonaliz it to obtained normal-modes and its frequencies. This is done using the phonon_calculator class available within elph_classes.

In [8]:
phonon = phonon_calculator(forces = forces,\
                           mass = mass,\
                           mode = 'FD',\
                           deltax = 0.005)

Please remember to use the same mode and deltax that was used to prepare the input files, i.e., on tutorial cart_phonon_1. 

Next, we would take the computed dynamical matrix and instantiate a dynamica matrix (dm) class available in elph_classes. This class has the methods to refine the dynamical matrix using the suitable acoustic sum rule (asr). Here we have a linear molecule so we will choose asr = 'lin'. For a non-linear polyatomic molecule and crystal, one should use asr = 'lin' & asr = 'crystal', respectively.

In [9]:
dm = dm(dynmat = phonon.dynmat, mass = mass)
dm.apply_asr(opt_coord = opt_coord, asr = 'lin')

Now it is time to write all information into a checkpoint/restart file that can be used for future reference. This file would be the starting point for calculating a normal-mode finite difference method, stochastic method, etc.

In [10]:
restart_file = write_pyepfd_info(inp_dynmat = None,\
                  dynmat = dm.dynmatrix,\
                  ref_dynmat = dm.refdynmatrix,\
                  mass = mass,atoms = atoms,\
                  opt_coord = opt_coord,\
                  cell = qbouts[0].cell,\
                  file_name='fdphonon.xml',\
                  mode='fd',\
                  deltax=0.005,\
                  asr = 'lin')
# deleting the restart file object to finish writing 
del restart_file

Time spent at write_info       0.0004 s


Let us have a look at the restart file fdphonon.xml.

In [11]:
%%bash
cat fdphonon.xml

<pyepfd>
  <phonon mode='fd'>
    <ngrid> 1 </ngrid>
    <deltax> 0.005 </deltax>
    <deltae> 0.001 </deltae>
    <asr> lin </asr>
    <dynmat shape='(9, 9)'>
[  2.12445535e-07,  3.42875298e-11,  1.71437649e-11,  1.33933949e-06,  0.00000000e+00, 
  -1.71437649e-11, -3.07800297e-06,  1.97867238e-11,  3.95734476e-11,  3.42875298e-11, 
   2.12445535e-07,  1.71437649e-11,  3.42875298e-11,  1.33932235e-06, -1.71437649e-11, 
   1.97867238e-11, -3.07800297e-06,  1.97867238e-11,  1.71437649e-11,  1.71437649e-11, 
   3.45437948e-05,  0.00000000e+00,  0.00000000e+00, -3.07092832e-06,  0.00000000e+00, 
   0.00000000e+00, -3.67259006e-05,  1.33933949e-06,  3.42875298e-11,  0.00000000e+00, 
   2.12274097e-07,  0.00000000e+00, -3.42875298e-11, -3.07790403e-06,  1.97867238e-11, 
  -3.95734476e-11,  0.00000000e+00,  1.33932235e-06,  0.00000000e+00,  0.00000000e+00, 
   2.12342672e-07,  1.71437649e-11,  0.00000000e+00, -3.07788425e-06, -1.97867238e-11, 
  -1.71437649e-11, -1.71437649e-11, -3.07092832e



We see this is an xml file.It can be used to gather all information for further pyEPFD calculations, for example a normal-mode Hessian calculation or stochastic electron-phonon calculation.

Sometime we would like to visualize the normal modes. For that purpose common visualization codes can be used and pyEPFD has methods to write normal-modes into an 'axsf', 'nmd' or 'molden' files that can be visualized with XCRYSDEN, VMD or MOLDEN programs, respectively. The below example shows how to do it using XCRYSDEN file as an example. 


In [12]:
nmodes = write_nmode(atoms = atoms, 
                     cell_v = cell_v, 
                     opt_coord = opt_coord, 
                     mode_v = dm.refV, # refined normal mode vectors after ASR 
                     mode_freq = dm.refomega, #refined frequencies 
                     fmt='axsf', #Options are: 'axsf','nmd','molden'
                     file_path='refdynmat')
# Deleting the nmodes object to finish printing the information into the file
del nmodes

In [13]:
%%bash
cat refdynmat.axsf

ANIMSTEPS 9
 CRYSTAL
 PRIMVEC
    10.583544980     0.000000000     0.000000000
     0.000000000    10.583544980     0.000000000
     0.000000000     0.000000000    10.583544980
PRIMCOORD  1
   3  1
     O                 0              0        1.15713     0.00543872   -0.000154545   -7.63158e-07
     O                 0              0       -1.15713    -0.00078962    9.78154e-06   -7.63158e-07
     C                 0              0   -1.58753e-06     0.00232455   -7.23814e-05   -7.63158e-07
PRIMCOORD  2
   3  1
     O                 0              0        1.15713    -1.5501e-05     0.00180638   -0.000434643
     O                 0              0       -1.15713     0.00502252   -0.000506981   -0.000434643
     C                 0              0   -1.58753e-06     0.00250351    0.000649697   -0.000434643
PRIMCOORD  3
   3  1
     O                 0              0        1.15713    0.000169877     0.00511544    1.96058e-05
     O                 0              0       -1.15713    -0

The refdynmat.axsf containes the normal modes after refining them applying acoustic sum rule. This file can be visualized with XCRYSDEN. If we choose fmt='nmd' / 'molden' that would create .nmd / .molden file respectively. They can be visualized with VMD and Molden, respectively. 